Coming to another question. Once you have fulltext as per both methods
(load it in separate columns), you can either give it back to me for
training the classifier or alternatively, you can take a crack
yourself. We are going to use multilabel classification.

I helped one of my other interns do this just a few days ago; you can
check it out here
https://github.com/SriyaNelluri/specrom_analytics/blob/main/Week3Mod.ipynb

In [1]:
import pandas as pd
from fake_useragent import UserAgent
import bs4
import justext
import time
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
pd.options.mode.chained_assignment = None 

In [2]:
def requests_retry_session(
    retries=5,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [3]:
df = pd.read_csv("../Data/iab_text_tiers.csv").fillna("")

In [4]:
df = df[['Unique ID', 'Parent', 'Name', 'Position', 'URL', 'Title', 'Snippet', 
         'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'with_bs4', 'with_justext']]

In [5]:
len(df)

11968

In [6]:
url = df.URL[0]
ua = UserAgent()

In [7]:
def with_bs4(html_response):
    soup = bs4.BeautifulSoup(html_response,'html.parser')
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()
    # print(text)
    return text.replace(',', '').replace('\n', ' ').replace('\t', '')[:10000]

In [8]:
def with_justext(html_response):
    paragraphs = justext.justext(html_response, justext.get_stoplist("English"))
    good_paragraph = ''
    i = 0
    for paragraph in paragraphs:
        if not paragraph.is_boilerplate:
            good_paragraph = good_paragraph + '  ' + paragraph.text
            i = i + 1
    # print(good_paragraph)
    return good_paragraph.replace(',', '').replace('\n', ' ').replace('\t', '')[:10000]

In [9]:
# for i in range(11000, 11500):
#     my_headers = {'User-Agent': ua.random}
#     url = df.URL.iloc[i]
#     try:
#         rr = requests_retry_session().get(url, timeout = 5, headers = my_headers)
#         # rr = requests.get(url, headers = my_headers)
#         ht_response = rr.text
#         df['with_bs4'][i] = with_bs4(ht_response)
#         df['with_justext'][i] = with_justext(ht_response)
#         print(i)
#     except:
#         print(i, "TLE")
#         df['with_bs4'][i] = "exceeded"
#         df['with_justext'][i] = "exceeded"
    

In [13]:
len(df[df.with_justext == "exceeded"])

218

In [11]:
for i in range(len(df)):
    if df.iloc[i].with_justext == "exceeded":
        my_headers = {'User-Agent': ua.random}
        url = df.URL.iloc[i]
        try:
            rr = requests_retry_session().get(url, timeout = 5, headers = my_headers)
            ht_response = rr.text
            df['with_bs4'][i] = with_bs4(ht_response)
            df['with_justext'][i] = with_justext(ht_response)
            print(i, "found text")
        except:    
            df['with_bs4'][i] = "exceeded"
            df['with_justext'][i] = "exceeded"
            print(i, "TLE")

27 TLE
35 TLE
44 found text
49 found text
63 found text
64 TLE
65 TLE
75 TLE
78 found text
85 TLE
115 TLE
117 TLE
143 TLE
144 TLE
147 TLE
149 TLE
178 TLE
333 found text
372 TLE
514 TLE
599 TLE
619 TLE
622 found text
640 TLE
642 TLE
688 TLE
691 found text
760 TLE
798 TLE
846 TLE
976 found text
982 found text
1035 found text
1105 TLE
1129 TLE
1217 TLE
1219 TLE
1234 found text
1248 TLE
1301 TLE
1306 TLE
1317 TLE
1408 TLE
1409 TLE
1432 found text
1483 TLE
1511 TLE
1575 TLE
1577 TLE
1687 TLE
1688 TLE
1741 TLE
1786 TLE
1932 TLE
1945 TLE
2019 TLE
2066 TLE
2178 TLE
2192 TLE
2194 TLE
2248 TLE
2338 TLE
2462 TLE
2468 TLE
2490 TLE
2530 TLE
2629 TLE
2658 TLE
2799 TLE
2827 TLE
2883 found text
2982 found text
3003 found text
3004 TLE
3008 TLE
3072 TLE
3082 found text
3085 found text
3087 found text
3088 found text
3089 found text
3090 found text
3091 found text
3092 found text
3107 TLE
3138 TLE
3153 found text
3372 TLE
3417 TLE
3463 found text
3487 found text
3544 found text
3602 found text
3660 foun

In [12]:
df.to_csv("../Data/iab_text_tiers1.csv")